In [2]:
import torch
import pandas as pd
import numpy as np
#import random
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold,StratifiedKFold 
import gc
import os
from os import path
from sys import path as systemPath
systemPath.append(path.join('..', '..'))
from scipy.spatial import distance_matrix
#import argparse
import random
import matplotlib.pyplot as plt
import networkx as nx
import MDAnalysis as mda
from matplotlib.ticker import MaxNLocator,MultipleLocator
from MDAnalysis.core.groups import AtomGroup
import seaborn as sns
import statistics as st
from scipy.stats import gaussian_kde
from sklearn.metrics import confusion_matrix, classification_report

c:\Users\Wendy\miniconda3\envs\collagen\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## dataset

In [3]:
# real position index
mutation_pos = [29, 29, 35, 35, 35, 41, 41, 50, 59, 62, 77, 86, 89, 92, 92, 95, 101, 101, 104, 107, 107, 110, 113, 113, 119, 119, 122, 122, 137, 137, 143, 143, 149, 149, 152, 161, 164, 164, 170, 170, 176, 176, 188, 191, 191, 194, 203, 203, 206, 209, 215, 218, 218, 221, 221, 227, 227, 227, 230, 233, 236, 236, 236, 239, 239, 242, 242, 245, 245, 248, 251, 260, 260, 263, 266, 269, 272, 293, 299, 302, 302, 314, 320, 329, 341, 365, 365, 368, 368, 371, 371, 392, 398, 398, 398, 404, 407, 425, 425, 431, 431, 431, 455, 464, 464, 467, 473, 476, 488, 494, 497, 524, 527, 539, 539, 542, 542, 557, 557, 566, 575, 575, 581, 581, 593, 596, 602, 605, 608, 611, 614, 617, 626, 626, 647, 653, 653, 659, 659, 662, 671, 674, 677, 680, 680, 683, 689, 692, 695, 698, 704, 704, 707, 713, 716, 725, 734, 755, 764, 782, 785, 788, 800, 812, 818, 839, 848, 860, 860, 860, 863, 866, 878, 884, 884, 887, 890, 896, 899, 899, 902, 914, 914, 917, 920, 923, 926, 929, 938, 944, 962, 980, 983, 983, 989, 989, 992, 1004, 1007, 1007, 1010, 1013, 1013, 1016, 1019, 1022, 1025, 116, 1028, 125, 131, 158, 164, 32, 32, 32, 179, 182, 191, 200, 35, 206, 206, 212, 218, 227, 38, 242, 41, 275, 275, 293, 308, 308, 329, 347, 50, 365, 374, 53, 386, 407, 410, 56, 416, 419, 431, 440, 443, 59, 59, 59, 452, 470, 473, 485, 506, 521, 524, 551, 560, 569, 572, 578, 581, 587, 596, 599, 602, 608, 662, 662, 665, 671, 671, 677, 686, 689, 692, 692, 701, 704, 86, 716, 719, 722, 734, 770, 812, 812, 815, 854, 101, 878, 893, 917, 932, 947, 971, 113, 995, 1091, 1097, 1103, 1106, 1139, 1163, 1166, 1166, 1175, 1175, 1184, 1190, 1220, 1226, 1229, 1229, 1238, 1247, 1247, 1253, 1259, 1259, 1262, 1262, 1268, 1271, 1280, 1283, 1289, 1295, 1298, 1301, 1301, 1301, 1304, 1304, 1307, 1310, 1310, 1310, 1310, 1313, 1313, 1319, 1319, 1322, 1322, 1331, 1331, 1337, 1340, 1343, 1343, 1349, 1349, 1358, 1361, 1364, 1364, 1379, 1382, 1391, 1391, 1397, 1400, 1406, 1418, 1421, 1427, 1433, 1442, 1451, 1451, 1457, 1457, 1463, 1466, 1472, 1481, 1484, 1484, 1484, 1490, 1493, 1499, 1499, 1502, 1505, 1514, 1517, 1517, 1520, 1526, 1535, 1559, 1565, 1568, 1568, 1571, 1574, 1574, 1589, 1601, 1607, 1610, 1613, 1619, 1634, 1634, 1637, 1643, 1649, 1649, 1649, 1655, 1673, 1673, 1685, 1685, 1688, 1691, 1694, 1703, 1709, 1712, 1718, 1724, 1730, 1736, 1739, 1739, 1742, 1745, 1751, 1757, 1757, 1760, 1763, 1766, 1769, 1769, 1778, 1802, 1808, 1820, 1826, 1832, 1850, 1853, 1865, 1868, 1874, 1877, 1889, 1910, 1919, 1922, 1928, 1928, 1943, 1946, 1946, 1955, 1961, 1970, 1979, 1985, 1988, 2000, 2000, 2009, 2012, 2015, 2024, 2030, 2039, 2051, 2054, 2057, 2063, 2069, 2072, 2075, 2075, 2075, 2075, 1163, 2063, 2063, 2066, 1166, 1169, 1172, 1172, 1172, 1184, 1190, 1220, 1232, 1238, 1241, 1241, 1244, 1250, 1256, 1262, 1262, 1277, 1277, 1085, 1286, 1289, 1292, 1310, 1088, 1313, 1325, 1328, 1358, 1370, 1094, 1373, 1415, 1424, 1427, 1100, 1454, 1490, 1523, 1535, 1538, 1541, 1544, 1112, 1598, 1607, 1619, 1634, 1634, 1646, 1673, 1682, 1688, 1706, 1712, 1718, 1721, 1727, 1733, 1736, 1742, 1745, 1760, 1763, 1769, 1775, 1775, 1778, 1784, 1787, 1790, 1793, 1796, 1814, 1820, 1826, 1829, 1847, 1142, 1853, 1907, 1916, 1916, 1928, 1934, 1955, 1955, 1958, 1958, 1964, 1967, 1976, 1997, 2003, 2018, 2057]
# mutation type
mutation_name = ["ALA" , "ASP", "CYS", "SER", "ASP", "ARG", "VAL", "ARG", "CYS", "CYS", "ASP", "ARG", "CYS", "ARG", "GLU", "ARG", "ARG", "VAL", "GLU", "SER", "VAL", "CYS", "ARG", "ASP", "SER", "ASP", "SER", "ALA", "ARG", "ASP", "SER", "ARG", "ARG", "ASP", "ARG", "GLU", "ARG", "ASP", "ARG", "ALA", "CYS", "SER", "ARG", "CYS", "SER", "CYS", "VAL", "ALA", "SER", "SER", "ALA", "CYS", "VAL", "CYS", "SER", "ARG", "CYS", "SER", "GLU", "SER", "CYS", "ALA", "ASP", "CYS", "ASP", "CYS", "SER", "SER", "CYS", "SER", "ALA", "CYS", "ALA", "SER", "CYS", "VAL", "VAL", "CYS", "SER", "ARG", "GLU", "ARG", "ALA", "CYS", "CYS", "CYS", "ALA", "CYS", "SER", "CYS", "ASP", "ALA", "ARG", "SER", "CYS", "ARG", "ARG", "SER", "ALA", "CYS", "SER", "ASP", "ARG", "CYS", "SER", "SER", "ARG", "VAL", "ARG", "SER", "ALA", "VAL", "SER", "CYS", "SER", "CYS", "SER", "SER", "ASP", "ARG", "CYS", "ASP", "SER", "VAL", "SER", "CYS", "VAL", "SER", "CYS", "SER", "SER", "SER", "CYS", "SER", "SER", "VAL", "ALA", "SER", "ARG", "CYS", "ALA", "ALA", "SER", "ARG", "ASP", "ARG", "ASP", "SER", "CYS", "ALA", "CYS", "SER", "CYS", "SER", "SER", "SER", "CYS", "GLU", "CYS", "CYS", "CYS", "ARG", "ASP", "ARG", "VAL", "CYS", "SER", "SER", "ALA", "VAL", "ARG", "SER", "SER", "SER", "CYS", "SER", "SER", "SER", "SER", "ASP", "ALA", "SER", "CYS", "SER", "CYS", "SER", "ALA", "SER", "ASP", "ALA", "CYS", "SER", "CYS", "ASP", "SER", "VAL", "ARG", "CYS", "SER", "CYS", "ASP", "CYS", "SER", "VAL", "SER", "VAL", "VAL", "GLU", "ASP", "SER", "VAL", "VAL", "SER", "ARG", "SER", "ALA", "CYS", "ASP", "ASP", "SER", "ARG", "VAL", "ASP", "ALA", "SER", "ALA", "VAL", "ALA", "SER", "CYS", "ASP", "ASP", "VAL", "ALA", "SER", "VAL", "GLU", "ASP", "ARG", "ARG", "ALA", "ASP", "ARG", "ASP", "SER", "ARG", "VAL", "ARG", "ASP", "ARG", "ASP", "SER", "ARG", "SER", "SER", "SER", "SER", "SER", "ALA", "SER", "SER", "VAL", "VAL", "ARG", "ASP", "SER", "SER", "ASP", "SER", "SER", "ARG", "ALA", "ASP", "ASP", "VAL", "ASP", "ARG", "VAL", "ARG", "VAL", "VAL", "ALA", "GLU", "ASP", "VAL", "SER", "ASP", "ASP", "ASP", "ALA", "ASP", "ARG", "GLU", "ARG", "ASP", "ARG", "SER", "SER", "ALA", "SER", "ASP", "ASP", "ASP", "ASP", "SER", "VAL", "VAL", "SER", "ASP", "ARG", "ASP", "ASP", "VAL", "ASP", "ASP", "ASP", "VAL", "ASP", "ASP", "VAL", "SER", "ALA", "VAL", "ALA", "VAL", "VAL", "ARG", "ASP", "ARG", "ARG", "SER", "GLU", "SER", "CYS", "ASP", "ASP", "VAL", "SER", "CYS", "SER", "ARG", "PRO", "ARG", "SER", "CYS", "SER", "SER", "CYS", "CYS", "SER", "SER", "TRP", "SER", "ALA", "ASP", "VAL", "GLU", "VAL", "CYS", "SER", "SER", "VAL", "SER", "ASP", "SER", "SER", "GLU", "ARG", "GLU", "CYS", "SER", "ALA", "ASP", "VAL", "VAL", "GLU", "GLU", "GLU", "VAL", "ASP", "SER", "ASP", "VAL", "ARG", "SER", "ARG", "GLU", "CYS", "GLU", "CYS", "SER", "VAL", "ARG", "ASP", "CYS", "ARG", "SER", "CYS", "SER", "VAL", "SER", "ASP", "SER", "ARG", "VAL", "ASP", "GLU", "VAL", "SER", "VAL", "ASP", "ASP", "ARG", "VAL", "ASP", "ASP", "CYS", "ASP", "SER", "ASP", "ASP", "SER", "SER", "CYS", "CYS", "ARG", "GLU", "SER", "VAL", "VAL", "SER", "VAL", "VAL", "SER", "SER", "ARG", "SER", "CYS", "ASP", "ARG", "SER", "ALA", "CYS", "SER", "SER", "ARG", "VAL", "SER", "CYS", "SER", "ASP", "ASP", "SER", "ARG", "ARG", "CYS", "ARG", "SER", "SER", "CYS", "ASP", "VAL", "ASP", "VAL", "VAL", "ASP", "SER", "SER", "ARG", "ARG", "GLU", "ARG", "ASP", "SER", "SER", "CYS", "ASP", "ASP", "VAL", "VAL", "CYS", "ALA", "ARG", "ARG", "SER", "ASP", "VAL", "GLU", "ASP", "VAL", "ASP", "ARG", "ASP", "ASP", "VAL", "CYS", "VAL", "GLU", "ARG", "SER", "ARG", "ARG", "GLU", "ARG", "ARG", "ARG", "ASP", "SER", "CYS", "SER", "VAL", "CYS", "SER", "ARG", "ASP", "ARG", "ASP", "ASP", "GLU", "ARG", "GLU", "ASP", "ARG", "ARG", "SER", "ALA", "ASP", "SER", "ASP", "SER", "VAL", "SER", "ASP", "ARG", "VAL", "ASP", "ASP", "CYS", "ASP", "CYS", "ASP", "ARG", "VAL", "SER", "CYS", "GLU", "ARG", "VAL", "CYS", "GLU", "ASP", "CYS", "VAL", "ASP", "ALA", "CYS", "SER", "VAL", "ARG", "ALA", "GLU", "VAL", "SER", "CYS", "SER", "SER", "ASP", "VAL", "ASP", "GLU", "ASP", "ASP", "GLU", "ARG", "ASP", "ASP", "ASP", "SER", "ARG", "SER", "VAL", "ASP", "ASP", "ARG", "ASP", "ASP", "CYS"]
# lethal:0, non-lethal:1 
lethal = [ 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1]  
# triple number 
triplet_number = [5, 5, 7, 7, 7, 9, 9, 12, 15, 16, 21, 24, 25, 26, 26, 27, 29, 29, 30, 31, 31, 32, 33, 33, 35, 35, 36, 36, 41, 41, 43, 43, 45, 45, 46, 49, 50, 50, 52, 52, 54, 54, 58, 59, 59, 60, 63, 63, 64, 65, 67, 68, 68, 69, 69, 71, 71, 71, 72, 73, 74, 74, 74, 75, 75, 76, 76, 77, 77, 78, 79, 82, 82, 83, 84, 85, 86, 93, 95, 96, 96, 100, 102, 105, 109, 117, 117, 118, 118, 119, 119, 126, 128, 128, 128, 130, 131, 137, 137, 139, 139, 139, 147, 150, 150, 151, 153, 154, 158, 160, 161, 170, 171, 175, 175, 176, 176, 181, 181, 184, 187, 187, 189, 189, 193, 194, 196, 197, 198, 199, 200, 201, 204, 204, 211, 213, 213, 215, 215, 216, 219, 220, 221, 222, 222, 223, 225, 226, 227, 228, 230, 230, 231, 233, 234, 237, 240, 247, 250, 256, 257, 258, 262, 266, 268, 275, 278, 282, 282, 282, 283, 284, 288, 290, 290, 291, 292, 294, 295, 295, 296, 300, 300, 301, 302, 303, 304, 305, 308, 310, 316, 322, 323, 323, 325, 325, 326, 330, 331, 331, 332, 333, 333, 334, 335, 336, 337, 34, 338, 37, 39, 48, 50, 6, 6, 6, 55, 56, 59, 62, 7, 64, 64, 66, 68, 71, 8, 76, 9, 87, 87, 93, 98, 98, 105, 111, 12, 117, 120, 13, 124, 131, 132, 14, 134, 135, 139, 142, 143, 15, 15, 15, 146, 152, 153, 157, 164, 169, 170, 179, 182, 185, 186, 188, 189, 191, 194, 195, 196, 198, 216, 216, 217, 219, 219, 221, 224, 225, 226, 226, 229, 230, 24, 234, 235, 236, 240, 252, 266, 266, 267, 280, 29, 288, 293, 301, 306, 311, 319, 33, 327, 10, 12, 14, 15, 26, 34, 35, 35, 38, 38, 41, 43, 53, 55, 56, 56, 59, 62, 62, 64, 66, 66, 67, 67, 69, 70, 73, 74, 76, 78, 79, 80, 80, 80, 81, 81, 82, 83, 83, 83, 83, 84, 84, 86, 86, 87, 87, 90, 90, 92, 93, 94, 94, 96, 96, 99, 100, 101, 101, 106, 107, 110, 110, 112, 113, 115, 119, 120, 122, 124, 127, 130, 130, 132, 132, 134, 135, 137, 140, 141, 141, 141, 143, 144, 146, 146, 147, 148, 151, 152, 152, 153, 155, 158, 166, 168, 169, 169, 170, 171, 171, 176, 180, 182, 183, 184, 186, 191, 191, 192, 194, 196, 196, 196, 198, 204, 204, 208, 208, 209, 210, 211, 214, 216, 217, 219, 221, 223, 225, 226, 226, 227, 228, 230, 232, 232, 233, 234, 235, 236, 236, 239, 247, 249, 253, 255, 257, 263, 264, 268, 269, 271, 272, 276, 283, 286, 287, 289, 289, 294, 295, 295, 298, 300, 303, 306, 308, 309, 313, 313, 316, 317, 318, 321, 323, 326, 330, 331, 332, 334, 336, 337, 338, 338, 338, 338, 34, 334, 334, 335, 35, 36, 37, 37, 37, 41, 43, 53, 57, 59, 60, 60, 61, 63, 65, 67, 67, 72, 72, 8, 75, 76, 77, 83, 9, 84, 88, 89, 99, 103, 11, 104, 118, 121, 122, 13, 131, 143, 154, 158, 159, 160, 161, 17, 179, 182, 186, 191, 191, 195, 204, 207, 209, 215, 217, 219, 220, 222, 224, 225, 227, 228, 233, 234, 236, 238, 238, 239, 241, 242, 243, 244, 245, 251, 253, 255, 256, 262, 27, 264, 282, 285, 285, 289, 291, 298, 298, 299, 299, 301, 302, 305, 312, 314, 319, 332]
# chain 
alpha12 = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

In [4]:
"amino acids clusters"
PRO = ["PRO","HYP"]
Small = ["GLY","ALA","CYS","SER","THR"]
Charged = ["HIS", "LYS", "ARG", "GLU","ASP"]
Hydrophobic = ["VAL", "ILE", "LEU", "MET", "PHE", "TYR" , "TRP"]
Polar = ["ASN" , "GLN"]
severe = ["VAL","ASP"]
moderate = ["GLU","CYS","ARG"]
mild = ["PRO","TRP","ALA","SER"]

## load

In [ ]:
##儲存資料
radius_all=np.load(f'D:/collagen/new_uh_rad/traj_radius_all.npy', allow_pickle = True)
radius_wt=np.load(f'D:/collagen/new_uh_rad/traj_radius_wt.npy', allow_pickle = True)
unit_hei_all=np.load(f'D:/collagen/new_uh_rad/traj_unit_hei.npy', allow_pickle = True)
unit_hei_wt=np.load(f'D:/collagen/new_uh_rad/traj_unit_hei_wt.npy', allow_pickle = True)

gpo_radius_traj = np.load(f'D:/collagen/gpo30/ana/gpo_traj_radius.npy', allow_pickle = True)
gpo_unit_hei_traj = np.load(f'D:/collagen/gpo30/ana/gpo_traj_unhei.npy', allow_pickle = True)

mutation_name_arr = np.load(f'D:/collagen/data/mutation_name.npy')  # (587,)
lethal_arr = np.load(f'D:/collagen/data/lethal.npy')  # (587,)
alpha12_arr = np.load(f'D:/collagen/data/alpha12.npy')  # (587,)
triplet_number_arr = np.load(f'D:/collagen/data/triplet_number.npy')  # (587,)

aa17_all = np.load("D:/collagen/amino_acid_type/aa17_class.npy")
aa17_all = np.char.upper(np.char.strip(aa17_all))   # 保險：去空白→大寫

FileNotFoundError: [Errno 2] No such file or directory: 'D:/collagen/amino_acid_type/aa17_class.npy'

## 計算frames平均與標準差

In [5]:
##平均與標準差
def calc_avg_std(arr):
    """
    對 3-D 陣列 arr 取 avg、std。
    回傳 shape = (arr.shape[0], arr.shape[2]) 的兩個 np.array
    """
    avg_out, std_out = [], []

    for i in range(arr.shape[0]):          # 對「第一軸」逐 slice
        avg_row, std_row = [], []
        for j in range(arr.shape[2]):      # 對「第三軸」逐 column
            vals = arr[i, :, j]            # 相當於原本的 arr[i,:,j]
            avg_row.append(st.mean(vals))
            std_row.append(st.stdev(vals)) # sample σ（與 st.dev 同義）
        avg_out.append(avg_row)
        std_out.append(std_row)

    return np.array(avg_out), np.array(std_out)
radius_avg,     radius_std     = calc_avg_std(radius_all)
radius_wt_avg,  radius_wt_std  = calc_avg_std(radius_wt)
unit_hei_avg,   unit_hei_std   = calc_avg_std(unit_hei_all)
unit_wt_avg,    unit_wt_std    = calc_avg_std(unit_hei_wt)

radius_diff_avg = radius_avg - radius_wt_avg
print(radius_diff_avg.shape)
unit_hei_diff_avg = unit_hei_avg - unit_wt_avg

(587, 9)


## all type (四張圖對比)

### rad分布圖

In [22]:
#radius 9pos分布圖
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
import os

def plot_kde_4groups(data_all, meta, title_prefix, t_tittle, alpha=1,
                     pos_labels=None, colors=None, save_path=None,
                     resname="CYS"):
    """
    data_all : (587, 9)   ← 每條突變 9 個位置的平均值
    meta     : dict，必含 mutation_name_arr, lethal_arr, alpha12_arr
    alpha    : 1 或 2 → 指定畫 α1 或 α2 資料
    title_prefix : e.g. "Unit Height" 或 "Radius"
    t_tittle      : "Mutation" 或 "WildType"
    save_path     : 圖檔儲存路徑（含檔名）
    resname       : 指定殘基名稱（如 CYS, SER, ARG）
    """

    if pos_labels is None:
        pos_labels = ["P4","P3","P2","P1","M0","P1′","P2′","P3′","P4′"]
    if colors is None:
        colors = ["#ff1493", "#ee7600", "#ffd700", "#00ffff",
                  "#000000", "#4575b4", "#542788", "#9e0142", "#00ff00"]

    # 0. 取出 meta 資訊
    mut  = meta["mutation_name_arr"].astype(str)
    leth = meta["lethal_arr"]
    a12  = meta["alpha12_arr"]

    # 1. 篩選指定殘基 + alpha 鏈別
    is_target = mut == resname
    is_alpha  = a12 == alpha
    mask = is_target & is_alpha

    data_sel = data_all[mask]
    leth_sel = leth[mask]

    # 2. lethal / nonlethal 遮罩
    bin_mask = {
        "nonlethal": leth_sel == 1,
        "lethal":    leth_sel == 0,
    }
    titles = [
        f"{resname}  α{alpha}  Nonlethal ({t_tittle})", 
        f"{resname}  α{alpha}  Lethal ({t_tittle})"
    ]

    # 3. 整理 datasets → list(2) × list(9)
    datasets = []
    for key in ["nonlethal", "lethal"]:
        sel = data_sel[bin_mask[key]]
        per_pos = [sel[:, i] for i in range(9)]
        datasets.append(per_pos)

    # 4. 取得 gpo KDE
    combined_vals = gpo_radius_traj[:, 1:].ravel()
    kde_c = gaussian_kde(combined_vals, bw_method=0.7)
    xs_c  = np.linspace(combined_vals.min()*0.9,
                        combined_vals.max()*1.1, 300)
    ys_c  = kde_c(xs_c)
    peak_idx = np.argmax(ys_c)
    peak_x   = xs_c[peak_idx]

    # 5. 畫圖
    fig, axes = plt.subplots(2, 1, figsize=(8, 7))
    axes = axes.ravel()
    for ax, data9, sub_title in zip(axes, datasets, titles):
        for i, samples in enumerate(data9):
            if samples.size == 0:
                continue
            kde = gaussian_kde(samples, bw_method=0.45)
            xs  = np.linspace(samples.min()*0.9, samples.max()*1.1, 300)
            ys  = kde(xs)
            ax.plot(xs, ys, color=colors[i], linewidth=1.0, label=pos_labels[i])
        ax.set_ylim(0, 1)
        ax.set_xlim(2.5, 9)

        ax.plot(xs_c, ys_c, color="purple", lw=0.5, label="gpo")
        ax.fill_between(xs_c, ys_c, linestyle='--', color="purple", alpha=0.1)
        ax.axvline(peak_x, color="purple", linestyle="--", linewidth=1)
        ax.text(peak_x, 0.906, f"{peak_x:.2f} Å", 
                color="purple", ha="center", va="bottom",
                fontsize=10, backgroundcolor="white")

        ax.set_title(sub_title, fontsize=11, fontweight='bold')
        ax.set_xlabel(f"{title_prefix} (Å)", fontsize=12)
        ax.set_ylabel("Density")
        ax.grid(True, alpha=.25)

    handles, labels = axes[0].get_legend_handles_labels()
    fig.legend(handles, labels, ncol=10, loc=(0.03, 0.89),
               frameon=False, fontsize=9)

    fig.suptitle(f"Distribution of {title_prefix} for nine Positions ({resname})",
                 fontsize=14)
    plt.tight_layout(rect=[0, 0, 1, 0.93])

    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches="tight")
    plt.close()

meta = {
    "mutation_name_arr": mutation_name_arr,
    "lethal_arr":    lethal_arr,
    "alpha12_arr":   alpha12_arr,
}

out_root = "D:/collagen/figures/a1_a2_WT_MUT"
os.makedirs(out_root, exist_ok=True)

for resname in ["SER", "ARG", "CYS", "GLU", "ASP", "VAL", "ALA" ]:
    out_dir = os.path.join(out_root, resname)
    os.makedirs(out_dir, exist_ok=True)

    plot_kde_4groups(radius_avg,    meta, "Radius", "Mutation", alpha=1,
                     save_path=os.path.join(out_dir, "radius_mutation_alpha1.png"),
                     resname=resname)

    plot_kde_4groups(radius_avg,    meta, "Radius", "Mutation", alpha=2,
                     save_path=os.path.join(out_dir, "radius_mutation_alpha2.png"),
                     resname=resname)

    plot_kde_4groups(radius_wt_avg, meta, "Radius", "WildType", alpha=1,
                     save_path=os.path.join(out_dir, "radius_wt_alpha1.png"),
                     resname=resname)

    plot_kde_4groups(radius_wt_avg, meta, "Radius", "WildType", alpha=2,
                     save_path=os.path.join(out_dir, "radius_wt_alpha2.png"),
                     resname=resname)


### uh分布圖


In [ ]:
#unit height 9pos分布圖
def plot_kde_4groups(data_all, meta, title_prefix, t_tittle, alpha=1,
                     pos_labels=None, colors=None, save_path=None,
                     resname="CYS"):
    """
    data_all : (587, 9)   ← 每條突變 9 個位置的平均值
    meta     : dict，必含 mutation_name_arr, lethal_arr, alpha12_arr
    alpha    : 1 或 2 → 指定畫 α1 或 α2 資料
    title_prefix : e.g. "Unit Height" 或 "unit_height"
    t_tittle      : "Mutation" 或 "WildType"
    save_path     : 圖檔儲存路徑（含檔名）
    resname       : 指定殘基名稱（如 CYS, SER, ARG）
    """

    if pos_labels is None:
        pos_labels = ["P4","P3","P2","P1","M0","P1′","P2′","P3′","P4′"]
    if colors is None:
        colors = ["#ff1493", "#ee7600", "#ffd700", "#00ffff",
                  "#000000", "#4575b4", "#542788", "#9e0142", "#00ff00"]

    # 0. 取出 meta 資訊
    mut  = meta["mutation_name_arr"].astype(str)
    leth = meta["lethal_arr"]
    a12  = meta["alpha12_arr"]

    # 1. 篩選指定殘基 + alpha 鏈別
    is_target = mut == resname
    is_alpha  = a12 == alpha
    mask = is_target & is_alpha

    data_sel = data_all[mask]
    leth_sel = leth[mask]

    # 2. lethal / nonlethal 遮罩
    bin_mask = {
        "nonlethal": leth_sel == 1,
        "lethal":    leth_sel == 0,
    }
    titles = [
        f"{resname}  α{alpha}  Nonlethal ({t_tittle})", 
        f"{resname}  α{alpha}  Lethal ({t_tittle})"
    ]

    # 3. 整理 datasets → list(2) × list(9)
    datasets = []
    for key in ["nonlethal", "lethal"]:
        sel = data_sel[bin_mask[key]]
        per_pos = [sel[:, i] for i in range(9)]
        datasets.append(per_pos)

    # 4. 取得 gpo KDE
    combined_vals = gpo_unit_hei_traj[:, 1:].ravel()
    kde_c = gaussian_kde(combined_vals, bw_method=0.7)
    xs_c  = np.linspace(combined_vals.min()*0.9,
                        combined_vals.max()*1.1, 300)
    ys_c  = kde_c(xs_c)
    peak_idx = np.argmax(ys_c)
    peak_x   = xs_c[peak_idx]

    # 5. 畫圖
    fig, axes = plt.subplots(2, 1, figsize=(8, 7))
    axes = axes.ravel()
    for ax, data9, sub_title in zip(axes, datasets, titles):
        for i, samples in enumerate(data9):
            if samples.size == 0:
                continue
            kde = gaussian_kde(samples, bw_method=0.45)
            xs  = np.linspace(samples.min()*0.9, samples.max()*1.1, 300)
            ys  = kde(xs)
            ax.plot(xs, ys, color=colors[i], linewidth=1.0, label=pos_labels[i])
        ax.set_ylim(0, 1)
        ax.set_xlim(3.5, 11.5)

        ax.plot(xs_c, ys_c, color="purple", lw=0.5, label="gpo")
        ax.fill_between(xs_c, ys_c, linestyle='--', color="purple", alpha=0.1)
        ax.axvline(peak_x, color="purple", linestyle="--", linewidth=1)
        ax.text(peak_x, 0.906, f"{peak_x:.2f} Å", 
                color="purple", ha="center", va="bottom",
                fontsize=10, backgroundcolor="white")

        ax.set_title(sub_title, fontsize=11, fontweight='bold')
        ax.set_xlabel(f"{title_prefix} (Å)", fontsize=12)
        ax.set_ylabel("Density")
        ax.grid(True, alpha=.25)

    handles, labels = axes[0].get_legend_handles_labels()
    fig.legend(handles, labels, ncol=10, loc=(0.03, 0.89),
               frameon=False, fontsize=9)

    fig.suptitle(f"Distribution of {title_prefix} for nine Positions ({resname})",
                 fontsize=14)
    plt.tight_layout(rect=[0, 0, 1, 0.93])

    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches="tight")
    plt.close()

meta = {
    "mutation_name_arr": mutation_name_arr,
    "lethal_arr":    lethal_arr,
    "alpha12_arr":   alpha12_arr,
}

out_root = "D:/collagen/figures/a1_a2_WT_MUT"
os.makedirs(out_root, exist_ok=True)

for resname in ["SER", "ARG", "CYS", "GLU", "ASP", "VAL", "ALA" ]:
    out_dir = os.path.join(out_root, resname)
    os.makedirs(out_dir, exist_ok=True)

    plot_kde_4groups(unit_hei_avg,    meta, "Unit Height", "Mutation", alpha=1,
                     save_path=os.path.join(out_dir, "unit_height_mutation_alpha1.png"),
                     resname=resname)

    plot_kde_4groups(unit_hei_avg,    meta, "Unit Height", "Mutation", alpha=2,
                     save_path=os.path.join(out_dir, "unit_height_mutation_alpha2.png"),
                     resname=resname)

    plot_kde_4groups(unit_wt_avg, meta, "Unit Height", "WildType", alpha=1,
                     save_path=os.path.join(out_dir, "unit_height_wt_alpha1.png"),
                     resname=resname)

    plot_kde_4groups(unit_wt_avg, meta, "Unit Height", "WildType", alpha=2,
                     save_path=os.path.join(out_dir, "unit_height_wt_alpha2.png"),
                     resname=resname)


## each position(9張圖)

### rad

In [8]:
def plot_kde_onepos_wt_vs_mut(data_mut, data_wt, meta,
                              pos_label="M0", t_title="Radius",
                              resname="ARG", alpha=1, save_path=None):
    """
    畫出單一突變種類（如 ARG）在特定位置的 KDE 分布圖（WT vs Mut, lethal vs nonlethal）
    """
    pos_labels_all = ["P4","P3","P2","P1","M0","P1′","P2′","P3′","P4′"]
    idx = pos_labels_all.index(pos_label)

    # 轉 ndarray & 篩選突變類型 + alpha 鏈
    mut_name = np.asarray(meta["mutation_name"], dtype=str)
    lethal   = np.asarray(meta["lethal_arr"])
    alpha12  = np.asarray(meta["alpha12_arr"])

    is_target  = mut_name == resname
    is_alpha   = alpha12 == alpha
    mask       = is_target & is_alpha

    if np.sum(mask) < 2:
        print(f"跳過 {resname} α{alpha} @ {pos_label}，資料太少")
        return

    mut_sel = data_mut[mask]
    wt_sel  = data_wt[mask]
    leth_sel = lethal[mask]

    mask_lethal    = leth_sel == 0
    mask_nonlethal = leth_sel == 1

    samples = dict(
        WT_L   = wt_sel[mask_lethal, idx],
        WT_N   = wt_sel[mask_nonlethal, idx],
        MUT_L  = mut_sel[mask_lethal, idx],
        MUT_N  = mut_sel[mask_nonlethal, idx],
    )

    # 取共同 KDE 範圍
    all_vals = np.concatenate([v for v in samples.values() if v.size >= 2])
    if all_vals.size < 2:
        print(f"跳過 {resname} α{alpha} @ {pos_label}，KDE 資料不足")
        return
    xs = np.linspace(all_vals.min()*0.9, all_vals.max()*1.1, 400)

    plt.figure(figsize=(8,3))

    style = dict(
        WT_L  = dict(color='tomato',      ls='--', lw=1, label='WT-Lethal'),
        WT_N  = dict(color='deepskyblue', ls='--', lw=1, label='WT-Nonlethal'),
        MUT_L = dict(color='#d62728',     lw=1, label='Mut-Lethal'),
        MUT_N = dict(color='royalblue',   lw=1, label='Mut-Nonlethal'),
    )

    for key, vals in samples.items():
        if vals.size < 2:
            continue
        kde = gaussian_kde(vals, 0.45)
        plt.plot(xs, kde(xs), **style[key])

    # 疊上 triplet 2–9 KDE
    combined_vals = gpo_radius_traj[:, 1:].ravel()
    kde_c = gaussian_kde(combined_vals, bw_method=0.7)
    xs_c  = np.linspace(combined_vals.min()*0.9,
                        combined_vals.max()*1.1, 300)
    ys_c  = kde_c(xs_c)
    plt.plot(xs_c, ys_c, color="purple", lw=0.5, label="gpo")
    plt.fill_between(xs_c, ys_c, linestyle='--', color="purple", alpha=0.1)
    peak_idx = np.argmax(ys_c)
    peak_x   = xs_c[peak_idx]
    plt.axvline(peak_x, color="purple", linestyle="--", linewidth=1)
    plt.text(peak_x, 0.9, f"{peak_x:.2f} Å",
             color="purple", ha="center", va="bottom",
             fontsize=10, backgroundcolor="white")

    plt.xlabel(f"{t_title} (Å)", fontsize=13)
    plt.ylabel("Density",        fontsize=13)
    plt.xlim(2.5, 9)
    plt.ylim(0, 1)
    plt.title(f"{resname} α{alpha} @ {pos_label} ({t_title})", fontsize=14)
    plt.legend(fontsize=9)
    plt.grid(True, alpha=.25)
    plt.tight_layout()

    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches="tight")
    plt.close()

meta = {
    "mutation_name": mutation_name_arr,
    "lethal_arr":    lethal_arr,
    "alpha12_arr":   alpha12_arr,
}

out_root = "D:/collagen/figures/1pos_wt_mut/radius"
os.makedirs(out_root, exist_ok=True)

resnames = np.unique(mutation_name_arr.astype(str))
positions = ["P4","P3","P2","P1","M0","P1′","P2′","P3′","P4′"]

for resname in resnames:
    for alpha in [1, 2]:
        for pos_label in positions:
            out_dir = os.path.join(out_root, resname, f"alpha{alpha}")
            os.makedirs(out_dir, exist_ok=True)

            fname = f"{resname}_alpha{alpha}_{pos_label}.png"
            fpath = os.path.join(out_dir, fname)

            plot_kde_onepos_wt_vs_mut(
                radius_avg,
                radius_wt_avg,
                meta,
                pos_label = pos_label,
                t_title   = "Radius",
                resname   = resname,
                alpha     = alpha,
                save_path = fpath
            )


NameError: name 'radius_avg' is not defined

### uh

In [ ]:
def plot_kde_onepos_wt_vs_mut(data_mut, data_wt, meta,
                              pos_label="M0", t_title="Unit Height",
                              resname="ARG", alpha=1, save_path=None):
    """
    畫出單一突變種類（如 ARG）在特定位置的 KDE 分布圖（WT vs Mut, lethal vs nonlethal）
    """
    pos_labels_all = ["P4","P3","P2","P1","M0","P1′","P2′","P3′","P4′"]
    idx = pos_labels_all.index(pos_label)

    # 轉 ndarray & 篩選突變類型 + alpha 鏈
    mut_name = np.asarray(meta["mutation_name"], dtype=str)
    lethal   = np.asarray(meta["lethal_arr"])
    alpha12  = np.asarray(meta["alpha12_arr"])

    is_target  = mut_name == resname
    is_alpha   = alpha12 == alpha
    mask       = is_target & is_alpha

    if np.sum(mask) < 2:
        print(f"跳過 {resname} α{alpha} @ {pos_label}，資料太少")
        return

    mut_sel = data_mut[mask]
    wt_sel  = data_wt[mask]
    leth_sel = lethal[mask]

    mask_lethal    = leth_sel == 0
    mask_nonlethal = leth_sel == 1

    samples = dict(
        WT_L   = wt_sel[mask_lethal, idx],
        WT_N   = wt_sel[mask_nonlethal, idx],
        MUT_L  = mut_sel[mask_lethal, idx],
        MUT_N  = mut_sel[mask_nonlethal, idx],
    )

    # 取共同 KDE 範圍
    all_vals = np.concatenate([v for v in samples.values() if v.size >= 2])
    if all_vals.size < 2:
        print(f"跳過 {resname} α{alpha} @ {pos_label}，KDE 資料不足")
        return
    xs = np.linspace(all_vals.min()*0.9, all_vals.max()*1.1, 400)

    plt.figure(figsize=(8,3))

    style = dict(
        WT_L  = dict(color='tomato',      ls='--', lw=1, label='WT-Lethal'),
        WT_N  = dict(color='deepskyblue', ls='--', lw=1, label='WT-Nonlethal'),
        MUT_L = dict(color='#d62728',     lw=1, label='Mut-Lethal'),
        MUT_N = dict(color='royalblue',   lw=1, label='Mut-Nonlethal'),
    )

    for key, vals in samples.items():
        if vals.size < 2:
            continue
        kde = gaussian_kde(vals, 0.45)
        plt.plot(xs, kde(xs), **style[key])

    # 疊上 triplet 2–9 KDE
    combined_vals = gpo_unit_hei_traj[:, 1:].ravel()
    kde_c = gaussian_kde(combined_vals, bw_method=0.7)
    xs_c  = np.linspace(combined_vals.min()*0.9,
                        combined_vals.max()*1.1, 300)
    ys_c  = kde_c(xs_c)
    plt.plot(xs_c, ys_c, color="purple", lw=0.5, label="gpo")
    plt.fill_between(xs_c, ys_c, linestyle='--', color="purple", alpha=0.1)
    peak_idx = np.argmax(ys_c)
    peak_x   = xs_c[peak_idx]
    plt.axvline(peak_x, color="purple", linestyle="--", linewidth=1)
    plt.text(peak_x, 0.9, f"{peak_x:.2f} Å",
             color="purple", ha="center", va="bottom",
             fontsize=10, backgroundcolor="white")

    plt.xlabel(f"{t_title} (Å)", fontsize=13)
    plt.ylabel("Density",        fontsize=13)
    plt.xlim(3.5, 11.5)
    plt.ylim(0, 1)
    plt.title(f"{resname} α{alpha} @ {pos_label} ({t_title})", fontsize=14)
    plt.legend(fontsize=9)
    plt.grid(True, alpha=.25)
    plt.tight_layout()

    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches="tight")
    plt.close()

meta = {
    "mutation_name": mutation_name_arr,
    "lethal_arr":    lethal_arr,
    "alpha12_arr":   alpha12_arr,
}

out_root = "D:/collagen/figures/1pos_wt_mut/unit_height"
os.makedirs(out_root, exist_ok=True)

resnames = np.unique(mutation_name_arr.astype(str))
positions = ["P4","P3","P2","P1","M0","P1′","P2′","P3′","P4′"]

for resname in resnames:
    for alpha in [1, 2]:
        for pos_label in positions:
            out_dir = os.path.join(out_root, resname, f"alpha{alpha}")
            os.makedirs(out_dir, exist_ok=True)

            fname = f"{resname}_alpha{alpha}_{pos_label}.png"
            fpath = os.path.join(out_dir, fname)

            plot_kde_onepos_wt_vs_mut(
                unit_hei_avg,
                unit_wt_avg,
                meta,
                pos_label = pos_label,
                t_title   = "Unit Height",
                resname   = resname,
                alpha     = alpha,
                save_path = fpath
            )


跳過 PRO α1 @ P4，資料太少
跳過 PRO α1 @ P3，資料太少
跳過 PRO α1 @ P2，資料太少
跳過 PRO α1 @ P1，資料太少
跳過 PRO α1 @ M0，資料太少
跳過 PRO α1 @ P1′，資料太少
跳過 PRO α1 @ P2′，資料太少
跳過 PRO α1 @ P3′，資料太少
跳過 PRO α1 @ P4′，資料太少
跳過 PRO α2 @ P4，資料太少
跳過 PRO α2 @ P3，資料太少
跳過 PRO α2 @ P2，資料太少
跳過 PRO α2 @ P1，資料太少
跳過 PRO α2 @ M0，資料太少
跳過 PRO α2 @ P1′，資料太少
跳過 PRO α2 @ P2′，資料太少
跳過 PRO α2 @ P3′，資料太少
跳過 PRO α2 @ P4′，資料太少
跳過 TRP α1 @ P4，資料太少
跳過 TRP α1 @ P3，資料太少
跳過 TRP α1 @ P2，資料太少
跳過 TRP α1 @ P1，資料太少
跳過 TRP α1 @ M0，資料太少
跳過 TRP α1 @ P1′，資料太少
跳過 TRP α1 @ P2′，資料太少
跳過 TRP α1 @ P3′，資料太少
跳過 TRP α1 @ P4′，資料太少
跳過 TRP α2 @ P4，資料太少
跳過 TRP α2 @ P3，資料太少
跳過 TRP α2 @ P2，資料太少
跳過 TRP α2 @ P1，資料太少
跳過 TRP α2 @ M0，資料太少
跳過 TRP α2 @ P1′，資料太少
跳過 TRP α2 @ P2′，資料太少
跳過 TRP α2 @ P3′，資料太少
跳過 TRP α2 @ P4′，資料太少


In [ ]:
print(aa17_all.shape)

(587, 17)


## decision tree

In [ ]:
# === 選取 a1-SER 的資料 ===
is_SER_a1 = (mutation_name_arr == "SER") & (alpha12_arr == 1)
aa_sel    = aa17_all[is_SER_a1]
true_label = lethal_arr[is_SER_a1]  # 0 = L, 1 = N

# === 分類規則 ===
def classify_a1_SER(tree_input):
    P4  = tree_input[0]
    P2  = tree_input[2]
    P1  = tree_input[3]
    P1p = tree_input[4]
    P2p = tree_input[5]

    if P4 in {"PRO", "SMALL", "CHARGED"}:
        if P1 in {"PRO", "SMALL", "CHARGED"}:
            return "N"
        elif P1 in {"POLAR", "HYDROPHOBIC"}:
            if P2 == "CHARGED":
                return "L"
            elif P2p in {"PRO", "SMALL", "HYDROPHOBIC"}:
                return "N"
            elif P2p == "CHARGED":
                return "L"
    elif P4 in {"POLAR", "HYDROPHOBIC"}:
        if P1 in {"POLAR", "CHARGED"}:
            return "N"
        elif P2p == "SMALL":
            return "N"
        elif P2p in {"PRO", "CHARGED"}:
            return "L"
    return "N"  # fallback 預設為非致死

# === 執行分類 ===
pred_label = np.array([0 if classify_a1_SER(x) == "L" else 1 for x in aa_sel])  # 0 = L, 1 = N

# === 評估結果 ===
confmat = confusion_matrix(true_label, pred_label)
report  = classification_report(true_label, pred_label, target_names=["Lethal", "Nonlethal"], digits=3)

# === 輸出 ===
print("Confusion Matrix:\n", confmat)
print("\nClassification Report:\n", report)


Confusion Matrix:
 [[ 1 25]
 [ 2 57]]

Classification Report:
               precision    recall  f1-score   support

      Lethal      0.333     0.038     0.069        26
   Nonlethal      0.695     0.966     0.809        59

    accuracy                          0.682        85
   macro avg      0.514     0.502     0.439        85
weighted avg      0.584     0.682     0.582        85



In [ ]:
# === 選出 alpha2-SER ===
mask = (mutation_name_arr == "SER") & (alpha12_arr == 2)
aa_sel      = aa17_all[mask]
lethal_sel  = lethal_arr[mask]

# === 各位置的 offset 對應 index（以 M0 為中心）===
POS_IDX = {
    "M0":   0,
    "P2":   2,
    "P1′": -1,
    "P2′": -2,
    "P4′": -4,
    "P5′": -5,
}

def classify_a2_SER(row):
    p4p = row[POS_IDX["P4′"]]
    if p4p == "POLAR":
        return "N"
    else:
        p5p = row[POS_IDX["P5′"]]
        if p5p in {"PRO", "SMALL", "CHARGED", "HYDROPHOBIC"}:
            if p5p in {"PRO", "CHARGED", "HYDROPHOBIC"}:
                return "N"
            else:
                p1p = row[POS_IDX["P1′"]]
                if p1p in {"SMALL", "POLAR"}:
                    if p1p in {"SMALL", "CHARGED"}:
                        return "N"
                    else:
                        p2 = row[POS_IDX["P2"]]
                        if p2 in {"PRO", "HYDROPHOBIC"}:
                            return "N"
                        else:
                            return "L"
                else:
                    return "L"
        else:
            return "L"

# === 執行分類 ===
y_pred = np.array([classify_a2_SER(row) for row in aa_sel])
y_true = np.where(lethal_sel == 0, "L", "N")

# === 評估結果 ===
print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))
print("\nClassification Report:\n", classification_report(y_true, y_pred))

Confusion Matrix:
 [[ 3  4]
 [25 31]]

Classification Report:
               precision    recall  f1-score   support

           L       0.11      0.43      0.17         7
           N       0.89      0.55      0.68        56

    accuracy                           0.54        63
   macro avg       0.50      0.49      0.43        63
weighted avg       0.80      0.54      0.62        63



In [ ]:
# === 選取 a1-ARG 的資料 ===
is_ARG_a1 = (mutation_name_arr == "ARG") & (alpha12_arr == 1)
aa_sel    = aa17_all[is_ARG_a1]
true_label = lethal_arr[is_ARG_a1]  # 0 = L, 1 = N

# === 分類規則 ===
def classify_a1_ARG(tree_input):
    P2   = tree_input[2]
    P1p  = tree_input[5]
    P2p  = tree_input[6]
    P4p  = tree_input[8]

    if P2 not in {"SMALL", "HYDROPHOBIC"}:  # → P1'
        if P1p in {"PRO", "POLAR"}:
            return "L"
        elif P2p in {"PRO", "SMALL"}:
            return "N"
        elif P2p in {"POLAR", "CHARGED"}:
            return "L"
    else:  # P4' path
        if P4p == "SMALL":
            return "L"
        elif P1p in {"PRO", "SMALL", "POLAR"}:
            return "N"
        elif P1p == "HYDROPHOBIC":
            return "L"
    return "N"  # fallback

# === 執行分類 ===
pred_label = np.array([0 if classify_a1_ARG(x) == "L" else 1 for x in aa_sel])  # 0 = L, 1 = N

# === 評估結果 ===
confmat = confusion_matrix(true_label, pred_label)
report  = classification_report(true_label, pred_label, target_names=["Lethal", "Nonlethal"], digits=3)

# === 輸出 ===
print("Confusion Matrix:\n", confmat)
print("\nClassification Report:\n", report)


Confusion Matrix:
 [[ 8 20]
 [ 4 15]]

Classification Report:
               precision    recall  f1-score   support

      Lethal      0.667     0.286     0.400        28
   Nonlethal      0.429     0.789     0.556        19

    accuracy                          0.489        47
   macro avg      0.548     0.538     0.478        47
weighted avg      0.570     0.489     0.463        47



## check 結論與實際差距(CYS)

In [ ]:
 # === 選取 a1-CYS 的資料 ===
is_CYS_a1 = (mutation_name_arr == "CYS") & (alpha12_arr == 1)
aa_sel       = aa17_all[is_CYS_a1]
true_label   = lethal_arr[is_CYS_a1]  # 0 = L, 1 = N
triplet_sel  = triplet_number_arr[is_CYS_a1]
radius_sel   = radius_avg[is_CYS_a1]
radius_wt_sel = radius_wt_avg[is_CYS_a1]
uh_sel       = unit_hei_avg[is_CYS_a1]
uh_wt_sel    = unit_wt_avg[is_CYS_a1]

# === 分類規則 ===
def classify_a1_CYS(tree_input):
    P5p = tree_input[10]
    P1  = tree_input[3]
    P4  = tree_input[0]
    P4p = tree_input[8]

    if P5p in {"SMALL", "HYDROPHOBIC"}:
        if P1 in {"SMALL", "POLAR"}:
            return "N"
        elif P1 in {"PRO", "CHARGED", "HYDROPHOBIC"}:
            return "L"
    elif P4 in {"PRO", "POLAR", "CHARGED"}:
        if P4p == "HYDROPHOBIC":
            return "N"
        elif P5p == "PRO":
            return "N"
        elif P5p in {"POLAR", "CHARGED"}:
            return "L"
    return "N"  # fallback

# === 執行分類 ===
pred_label = np.array([0 if classify_a1_CYS(x) == "L" else 1 for x in aa_sel])  # 0 = L, 1 = N

# === 評估結果 ===
confmat = confusion_matrix(true_label, pred_label)
report  = classification_report(true_label, pred_label, target_names=["Lethal", "Nonlethal"], digits=3)

# === 匯出成 Excel 資料表 ===
records = []
for i in range(len(aa_sel)):
    mut_name = "CYS"
    triplet  = triplet_sel[i]
    true_lbl = "L" if true_label[i] == 0 else "N"
    pred_lbl = "L" if pred_label[i] == 0 else "N"
    correct  = "✔" if true_lbl == pred_lbl else "✘"

    # Radius & UH 差值（Mut - WT）
    rad_diff = radius_sel[i] - radius_wt_sel[i]
    uh_diff  = uh_sel[i] - uh_wt_sel[i]

    row = {
        "Triplet": triplet,
        "Mutation": mut_name,
        "True_Label": true_lbl,
        "Predicted": pred_lbl,
        "Correct?": correct
    }
    # 加入 radius 和 unit height 差值（位置為 P4 – P4′）
    for j, pos in enumerate(["P4", "P3", "P2", "P1", "M0", "P1′", "P2′", "P3′", "P4′"]):
        row[f"ΔRadius_{pos}"] = rad_diff[j]
        row[f"ΔUnitHei_{pos}"] = uh_diff[j]

    records.append(row)

# 建立 DataFrame 並輸出 Excel
df = pd.DataFrame(records)
# 欄位排序：先 meta，再 radius，再 unit height
meta_cols     = ["Triplet", "Mutation", "True_Label", "Predicted", "Correct?"]
radius_cols   = [col for col in df.columns if col.startswith("ΔRadius_")]
unithei_cols  = [col for col in df.columns if col.startswith("ΔUnitHei_")]

# 重新排列欄位順序
df = df[meta_cols + radius_cols + unithei_cols]
# === 存成 Excel 檔 ===
output_path = "D:/collagen/code/decision_results/a1_CYS_result.xlsx"

df.to_excel(output_path, index=False)
print(f"已儲存：{output_path}")

已儲存：D:/collagen/code/decision_results/a1_CYS_result.xlsx
